# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 2 2023 11:55AM,256520,10,SNAP22231,"Snap Medical Industries, LLC",Released
1,Feb 2 2023 11:55AM,256519,20,ATR-I000095,"HVL, LLC dba Atrium Innovations",Released
2,Feb 2 2023 11:54AM,256518,12,9107578,"SD Head USA, LLC",Released
3,Feb 2 2023 11:50AM,256515,12,HH8757,Hush Hush,Executing
4,Feb 2 2023 11:37AM,256513,12,9107562,"SD Head USA, LLC",Released
5,Feb 2 2023 11:24AM,256507,19,GP02062023,"Granules Pharmaceuticals, Inc.",Released
6,Feb 2 2023 11:18AM,256505,21,640245,"NBTY Global, Inc.",Released
7,Feb 2 2023 11:13AM,256504,10,MSP218136,"Methapharm, Inc.",Released
8,Feb 2 2023 11:13AM,256504,10,MSP218147,"Methapharm, Inc.",Released
9,Feb 2 2023 11:13AM,256504,10,MSP218154,"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
36,256513,Released,1
37,256515,Executing,1
38,256518,Released,1
39,256519,Released,1
40,256520,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
256513,NaN,NaN,1.0
256515,NaN,1.0,NaN
256518,NaN,NaN,1.0
256519,NaN,NaN,1.0
256520,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
256401,0.0,1.0,0.0
256414,0.0,1.0,0.0
256418,3.0,3.0,5.0
256427,0.0,0.0,1.0
256437,0.0,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
256401,0,1,0
256414,0,1,0
256418,3,3,5
256427,0,0,1
256437,0,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,256401,0,1,0
1,256414,0,1,0
2,256418,3,3,5
3,256427,0,0,1
4,256437,0,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,256401,,1,
1,256414,,1,
2,256418,3,3,5
3,256427,,,1
4,256437,,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 2 2023 11:55AM,256520,10,"Snap Medical Industries, LLC"
1,Feb 2 2023 11:55AM,256519,20,"HVL, LLC dba Atrium Innovations"
2,Feb 2 2023 11:54AM,256518,12,"SD Head USA, LLC"
3,Feb 2 2023 11:50AM,256515,12,Hush Hush
4,Feb 2 2023 11:37AM,256513,12,"SD Head USA, LLC"
5,Feb 2 2023 11:24AM,256507,19,"Granules Pharmaceuticals, Inc."
6,Feb 2 2023 11:18AM,256505,21,"NBTY Global, Inc."
7,Feb 2 2023 11:13AM,256504,10,"Methapharm, Inc."
10,Feb 2 2023 10:47AM,256497,16,"Emersa Waterbox, LLC"
11,Feb 2 2023 10:43AM,256495,19,ACG North America LLC


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Feb 2 2023 11:55AM,256520,10,"Snap Medical Industries, LLC",,,1
1,Feb 2 2023 11:55AM,256519,20,"HVL, LLC dba Atrium Innovations",,,1
2,Feb 2 2023 11:54AM,256518,12,"SD Head USA, LLC",,,1
3,Feb 2 2023 11:50AM,256515,12,Hush Hush,,1,
4,Feb 2 2023 11:37AM,256513,12,"SD Head USA, LLC",,,1
5,Feb 2 2023 11:24AM,256507,19,"Granules Pharmaceuticals, Inc.",,,1
6,Feb 2 2023 11:18AM,256505,21,"NBTY Global, Inc.",,,1
7,Feb 2 2023 11:13AM,256504,10,"Methapharm, Inc.",,,3
8,Feb 2 2023 10:47AM,256497,16,"Emersa Waterbox, LLC",,,1
9,Feb 2 2023 10:43AM,256495,19,ACG North America LLC,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 2 2023 11:55AM,256520,10,"Snap Medical Industries, LLC",1,,
1,Feb 2 2023 11:55AM,256519,20,"HVL, LLC dba Atrium Innovations",1,,
2,Feb 2 2023 11:54AM,256518,12,"SD Head USA, LLC",1,,
3,Feb 2 2023 11:50AM,256515,12,Hush Hush,,1,
4,Feb 2 2023 11:37AM,256513,12,"SD Head USA, LLC",1,,
5,Feb 2 2023 11:24AM,256507,19,"Granules Pharmaceuticals, Inc.",1,,
6,Feb 2 2023 11:18AM,256505,21,"NBTY Global, Inc.",1,,
7,Feb 2 2023 11:13AM,256504,10,"Methapharm, Inc.",3,,
8,Feb 2 2023 10:47AM,256497,16,"Emersa Waterbox, LLC",1,,
9,Feb 2 2023 10:43AM,256495,19,ACG North America LLC,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 2 2023 11:55AM,256520,10,"Snap Medical Industries, LLC",1,,
1,Feb 2 2023 11:55AM,256519,20,"HVL, LLC dba Atrium Innovations",1,,
2,Feb 2 2023 11:54AM,256518,12,"SD Head USA, LLC",1,,
3,Feb 2 2023 11:50AM,256515,12,Hush Hush,,1,
4,Feb 2 2023 11:37AM,256513,12,"SD Head USA, LLC",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 2 2023 11:55AM,256520,10,"Snap Medical Industries, LLC",1.0,NaN,NaN
1,Feb 2 2023 11:55AM,256519,20,"HVL, LLC dba Atrium Innovations",1.0,NaN,NaN
2,Feb 2 2023 11:54AM,256518,12,"SD Head USA, LLC",1.0,NaN,NaN
3,Feb 2 2023 11:50AM,256515,12,Hush Hush,NaN,1.0,NaN
4,Feb 2 2023 11:37AM,256513,12,"SD Head USA, LLC",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 2 2023 11:55AM,256520,10,"Snap Medical Industries, LLC",1.0,0.0,0.0
1,Feb 2 2023 11:55AM,256519,20,"HVL, LLC dba Atrium Innovations",1.0,0.0,0.0
2,Feb 2 2023 11:54AM,256518,12,"SD Head USA, LLC",1.0,0.0,0.0
3,Feb 2 2023 11:50AM,256515,12,Hush Hush,0.0,1.0,0.0
4,Feb 2 2023 11:37AM,256513,12,"SD Head USA, LLC",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3334120,54.0,7.0,3.0
12,1538817,3.0,3.0,0.0
15,1025878,30.0,29.0,0.0
16,256497,1.0,0.0,0.0
19,1538814,5.0,1.0,0.0
20,256519,1.0,0.0,0.0
21,1282434,5.0,0.0,0.0
22,256487,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3334120,54.0,7.0,3.0
1,12,1538817,3.0,3.0,0.0
2,15,1025878,30.0,29.0,0.0
3,16,256497,1.0,0.0,0.0
4,19,1538814,5.0,1.0,0.0
5,20,256519,1.0,0.0,0.0
6,21,1282434,5.0,0.0,0.0
7,22,256487,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,54.0,7.0,3.0
1,12,3.0,3.0,0.0
2,15,30.0,29.0,0.0
3,16,1.0,0.0,0.0
4,19,5.0,1.0,0.0
5,20,1.0,0.0,0.0
6,21,5.0,0.0,0.0
7,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,54.0
1,12,Released,3.0
2,15,Released,30.0
3,16,Released,1.0
4,19,Released,5.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21,22
Status,,,,,,,,
Completed,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Executing,7.0,3.0,29.0,0.0,1.0,0.0,0.0,0.0
Released,54.0,3.0,30.0,1.0,5.0,1.0,5.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21,22
0,Completed,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,7.0,3.0,29.0,0.0,1.0,0.0,0.0,0.0
2,Released,54.0,3.0,30.0,1.0,5.0,1.0,5.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21,22
0,Completed,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,7.0,3.0,29.0,0.0,1.0,0.0,0.0,0.0
2,Released,54.0,3.0,30.0,1.0,5.0,1.0,5.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()